For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:


In [1]:
# importing libraries
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

from geopy.geocoders import Nominatim 

import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

import folium 



In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [3]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [4]:
!pip install --user folium

In [5]:
# find table
soup.find('table').find_all('tr')

# find  rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [6]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [7]:

# create a new DataFrame from the stored data of list
toronto = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto.head()

PostalCode           Borough      Neighborhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront

 To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
# dropping not assigned value
toronto_dropna = toronto[toronto.Borough != "Not assigned"].reset_index(drop=True)
toronto_dropna.head()

PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

In [9]:
# group neighborhoods in the same borough
toronto_grouped = toronto_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_grouped.head()

PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Rouge, Malvern
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [10]:
# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in toronto_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_grouped.head()

PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Rouge, Malvern
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae

In [11]:
#  shape of clean data
toronto_grouped.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:


Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [12]:
# load the coordinates  csv
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [13]:
# Renaming column postal code for merging
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476

In [14]:
# Now merging 
toronto_new = toronto_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_new.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [15]:
#  Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="Coursera Capstone")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_new['Latitude'], toronto_new['Longitude'], toronto_new['Borough'], toronto_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [17]:
# filter borough names that contain the word Toronto
borough_names = list(toronto_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [18]:
# create a new DataFrame with only boroughs that contain the word Toronto
toronto_new = toronto_new[toronto_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_new.shape)
toronto_new.head()

(38, 5)


PostalCode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_new['Latitude'], toronto_new['Longitude'], toronto_new['Borough'], toronto_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [20]:
CLIENT_ID = '2ZA2NVZXQTI4DWWBTAC2UOE1V3HE3IRMFIDBSPQFR2Q1COVT' # your Foursquare ID
CLIENT_SECRET = 'YE0X2KY0PUWBWYGUU0ETQ3RHBCC4I5Z5HX1GQE3LQXCTKQJN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2ZA2NVZXQTI4DWWBTAC2UOE1V3HE3IRMFIDBSPQFR2Q1COVT
CLIENT_SECRET:YE0X2KY0PUWBWYGUU0ETQ3RHBCC4I5Z5HX1GQE3LQXCTKQJN


In [21]:
# Now, let's get the top 100 venues that are within a radius of 500 meters.
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_new['Latitude'], toronto_new['Longitude'], toronto_new['PostalCode'], toronto_new['Borough'], toronto_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [22]:
# convert the venues list into a new DataFrame
venues = pd.DataFrame(venues)

# define the column names
venues.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues.shape)
venues.head()

(1706, 9)


PostalCode       Borough Neighborhood  BoroughLatitude  BoroughLongitude  \
0        M4E  East Toronto  The Beaches        43.676357        -79.293031   
1        M4E  East Toronto  The Beaches        43.676357        -79.293031   
2        M4E  East Toronto  The Beaches        43.676357        -79.293031   
3        M4E  East Toronto  The Beaches        43.676357        -79.293031   
4        M4E  East Toronto  The Beaches        43.676357        -79.293031   

                            VenueName  VenueLatitude  VenueLongitude  \
0                   Glen Manor Ravine      43.676821      -79.293942   
1  The Big Carrot Natural Food Market      43.678879      -79.297734   
2                 Grover Pub and Grub      43.679181      -79.297215   
3                 Glen Stewart Ravine      43.676300      -79.294784   
4                       Upper Beaches      43.680563      -79.292869   

          VenueCategory  
0                 Trail  
1     Health Food Store  
2                   Pub  
3  Other Great Outdoors  
4          Neighborhood

In [23]:
# no of venues returned for potal code
venues.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

BoroughLatitude  \
PostalCode Borough          Neighborhood                                                          
M4E        East Toronto     The Beaches                                                       5   
M4K        East Toronto     The Danforth West, Riverdale                                     42   
M4L        East Toronto     The Beaches West, India Bazaar                                   21   
M4M        East Toronto     Studio District                                                  38   
M4N        Central Toronto  Lawrence Park                                                     4   
M4P        Central Toronto  Davisville North                                                  9   
M4R        Central Toronto  North Toronto West                                               20   
M4S        Central Toronto  Davisville                                                       33   
M4T        Central Toronto  Moore Park, Summerhill East                                       3   
M4V        Central Toronto  Deer Park, Forest Hill SE, Rathnelly, South Hil...               15   
M4W        Downtown Toronto Rosedale                                                          5   
M4X        Downtown Toronto Cabbagetown, St. James Town                                      41   
M4Y        Downtown Toronto Church and Wellesley                                             88   
M5A        Downtown Toronto Harbourfront, Regent Park                                        50   
M5B        Downtown Toronto Ryerson, Garden District                                        100   
M5C        Downtown Toronto St. James Town                                                  100   
M5E        Downtown Toronto Berczy Park                                                      57   
M5G        Downtown Toronto Central Bay Street                                               83   
M5H        Downtown Toronto Adelaide, King, Richmond                                        100   
M5J        Downtown Toronto Harbourfront East, Toronto Islands, Union Station               100   
M5K        Downtown Toronto Design Exchange, Toronto Dominion Centre                        100   
M5L        Downtown Toronto Commerce Court, Victoria Hotel                                  100   
M5N        Central Toronto  Roselawn                                                          1   
M5P        Central Toronto  Forest Hill North, Forest Hill West                               4   
M5R        Central Toronto  The Annex, North Midtown, Yorkville                              23   
M5S        Downtown Toronto Harbord, University of Toronto                                   35   
M5T        Downtown Toronto Chinatown, Grange Park, Kensington Market                       100   
M5V        Downtown Toronto CN Tower, Bathurst Quay, Island airport, Harbou...               14   
M5W        Downtown Toronto Stn A PO Boxes 25 The Esplanade                                 100   
M5X        Downtown Toronto First Canadian Place, Underground city                          100   
M6G        Downtown Toronto Christie                                                         15   
M6H        West Toronto     Dovercourt Village, Dufferin                                     14   
M6J        West Toronto     Little Portugal, Trinity                                         68   
M6K        West Toronto     Brockton, Exhibition Place, Parkdale Village                     24   
M6P        West Toronto     High Park, The Junction South                                    25   
M6R        West Toronto     Parkdale, Roncesvalles                                           15   
M6S        West Toronto     Runnymede, Swansea                                               36   
M7Y        East Toronto     Business Reply Mail Processing Centre 969 Eastern                18   

                                                                                BoroughLongitude  \
PostalCode Borough 

In [24]:
print('There are {} uniques categories.'.format(len(venues['VenueCategory'].unique())))

There are 235 uniques categories.


In [25]:
venues['VenueCategory'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Other Great Outdoors',
       'Neighborhood', 'Greek Restaurant', 'Ice Cream Shop',
       'Cosmetics Shop', 'Italian Restaurant', 'Brewery', 'Yoga Studio',
       'Restaurant', 'Fruit & Vegetable Store', 'Dessert Shop',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Indian Restaurant',
       'Furniture / Home Store', 'Grocery Store', 'Spa', 'Diner',
       'Bubble Tea Shop', 'Coffee Shop', 'Caribbean Restaurant', 'Bakery',
       'Sports Bar', 'Café', 'American Restaurant', 'Liquor Store', 'Gym',
       'Burger Joint', 'Fish & Chips Shop', 'Sushi Restaurant', 'Park',
       'Pet Store', 'Burrito Place', 'Steakhouse', 'Movie Theater',
       'Fast Food Restaurant', 'Sandwich Place', 'Intersection',
       'Food & Drink Shop', 'Fish Market', 'Gay Bar', 'Cheese Shop',
       'Chinese Restaurant', 'Middle Eastern Restaurant',
       'Thai Restaurant', 'Comfort Food Restaurant'], dtype=object)

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues['PostalCode'] 
toronto_onehot['Borough'] = venues['Borough'] 
toronto_onehot['Neighborhoods'] = venues['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1706, 238)


PostalCode       Borough Neighborhoods  Afghan Restaurant  Airport  \
0        M4E  East Toronto   The Beaches                  0        0   
1        M4E  East Toronto   The Beaches                  0        0   
2        M4E  East Toronto   The Beaches                  0        0   
3        M4E  East Toronto   The Beaches                  0        0   
4        M4E  East Toronto   The Beaches                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Auto Workshop  \
0           0                    0                 0              0   
1           0                    0                 0              0   
2           0                    0                 0              0   
3           0                    0                 0              0   
4           0                    0                 0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  \
0          0           0           0       0     0    0                 0   
1          0           0           0       0     0    0                 0   
2          0           0           0       0     0    0                 0   
3          0           0           0       0     0    0                 0   
4          0           0           0       0     0    0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0       0              0          0         0   
1                   0       0              0          0         0   
2                   0       0              0          0         0   
3                   0       0              0          0         0   
4                   0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Building  \
0                     0               0        0                0         0   
1                     0               0        0                0         0   
2                     0               0        0                0         0   
3                     0               0        0                0         0   
4                     0               0        0                0         0   

   Burger Joint  Burrito Place  Bus Line  Business Service  Butcher  Café  \
0             0              0         0                 0        0     0   
1             0              0         0                 0        0     0   
2             0              0         0                 0        0     0   
3             0              0         0                 0        0     0   
4             0              0         0                 

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues['PostalCode'] 
toronto_onehot['Borough'] = venues['Borough'] 
toronto_onehot['Neighborhoods'] = venues['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1706, 238)


PostalCode       Borough Neighborhoods  Afghan Restaurant  Airport  \
0        M4E  East Toronto   The Beaches                  0        0   
1        M4E  East Toronto   The Beaches                  0        0   
2        M4E  East Toronto   The Beaches                  0        0   
3        M4E  East Toronto   The Beaches                  0        0   
4        M4E  East Toronto   The Beaches                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Auto Workshop  \
0           0                    0                 0              0   
1           0                    0                 0              0   
2           0                    0                 0              0   
3           0                    0                 0              0   
4           0                    0                 0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  \
0          0           0           0       0     0    0                 0   
1          0           0           0       0     0    0                 0   
2          0           0           0       0     0    0                 0   
3          0           0           0       0     0    0                 0   
4          0           0           0       0     0    0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Boutique  \
0                   0       0              0          0         0   
1                   0       0              0          0         0   
2                   0       0              0          0         0   
3                   0       0              0          0         0   
4                   0       0              0          0         0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bubble Tea Shop  Building  \
0                     0               0        0                0         0   
1                     0               0        0                0         0   
2                     0               0        0                0         0   
3                     0               0        0                0         0   
4                     0               0        0                0         0   

   Burger Joint  Burrito Place  Bus Line  Business Service  Butcher  Café  \
0             0              0         0                 0        0     0   
1             0              0         0                 0        0     0   
2             0              0         0                 0        0     0   
3             0              0         0                 0        0     0   
4             0              0         0                 

In [29]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 238)


PostalCode           Borough  \
0         M4E      East Toronto   
1         M4K      East Toronto   
2         M4L      East Toronto   
3         M4M      East Toronto   
4         M4N   Central Toronto   
5         M4P   Central Toronto   
6         M4R   Central Toronto   
7         M4S   Central Toronto   
8         M4T   Central Toronto   
9         M4V   Central Toronto   
10        M4W  Downtown Toronto   
11        M4X  Downtown Toronto   
12        M4Y  Downtown Toronto   
13        M5A  Downtown Toronto   
14        M5B  Downtown Toronto   
15        M5C  Downtown Toronto   
16        M5E  Downtown Toronto   
17        M5G  Downtown Toronto   
18        M5H  Downtown Toronto   
19        M5J  Downtown Toronto   
20        M5K  Downtown Toronto   
21        M5L  Downtown Toronto   
22        M5N   Central Toronto   
23        M5P   Central Toronto   
24        M5R   Central Toronto   
25        M5S  Downtown Toronto   
26        M5T  Downtown Toronto   
27        M5V  Downtown Toronto   
28        M5W  Downtown Toronto   
29        M5X  Downtown Toronto   
30        M6G  Downtown Toronto   
31        M6H      West Toronto   
32        M6J      West Toronto   
33        M6K      West Toronto   
34        M6P      West Toronto   
35        M6R      West Toronto   
36        M6S      West Toronto   
37        M7Y      East Toronto   

                                        Neighborhoods  Afghan Restaurant  \
0                                         The Beaches           0.000000   
1                        The Danforth West, Riverdale           0.000000   
2                      The Beaches West, India Bazaar           0.000000   
3                                     Studio District           0.000000   
4                                       Lawrence Park           0.000000   
5                                    Davisville North           0.000000   
6                                  North Toronto West           0.000000   
7                                          Davisville           0.000000   
8                         Moore Park, Summerhill East           0.000000   
9   Deer Park, Forest Hill SE, Rathnelly, South Hi...           0.000000   
10                                           Rosedale           0.000000   
11                        Cabbagetown, St. James Town           0.000000   
12                               Church and Wellesley           0.011364   
13                          Harbourfront, Regent Park           0.000000   
14                           Ryerson, Garden District           0.000000   
15                                     St. James Town           0.000000   
16                                        Berczy Park           0.000000   
17                                 Central Bay Street           0.000000   
18                           Adelaide, King, Richmond           0.000000   
19  Harbourfront East, Toronto Islands, Union Station           0.000000   
20           Design Exchange, Toronto Dominion Centre           0.000000   
21                     Commerce Court, Victoria Hotel           0.000000   
22                                           Roselawn           0.000000   
23                Forest Hill North, Forest Hill West           0.000000   
24                The Annex, North Midtown, Yorkville           0.000000   
25                     Harbord, University of Toronto           0.000000   
26          Chinatown, Grange Park, Kensington Market           0.000000   
27  CN Tower, Bathurst Quay, Island airport, Harbo...           0.000000   
28                    Stn A PO Boxes 25 The Esplanade           0.000000   
29             First Canadian Place, Underground city           0.000000   
30                                           Christie           0.000000   
31                       Dovercourt Village, Dufferin           0.000000   
32                           Little Portugal, Trinity           0.000000   
33       Brockton, Exhibition Place, Parkdale Village

In [30]:
# display to 10 venues of each postal code
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted


(38, 13)


PostalCode           Borough  \
0         M4E      East Toronto   
1         M4K      East Toronto   
2         M4L      East Toronto   
3         M4M      East Toronto   
4         M4N   Central Toronto   
5         M4P   Central Toronto   
6         M4R   Central Toronto   
7         M4S   Central Toronto   
8         M4T   Central Toronto   
9         M4V   Central Toronto   
10        M4W  Downtown Toronto   
11        M4X  Downtown Toronto   
12        M4Y  Downtown Toronto   
13        M5A  Downtown Toronto   
14        M5B  Downtown Toronto   
15        M5C  Downtown Toronto   
16        M5E  Downtown Toronto   
17        M5G  Downtown Toronto   
18        M5H  Downtown Toronto   
19        M5J  Downtown Toronto   
20        M5K  Downtown Toronto   
21        M5L  Downtown Toronto   
22        M5N   Central Toronto   
23        M5P   Central Toronto   
24        M5R   Central Toronto   
25        M5S  Downtown Toronto   
26        M5T  Downtown Toronto   
27        M5V  Downtown Toronto   
28        M5W  Downtown Toronto   
29        M5X  Downtown Toronto   
30        M6G  Downtown Toronto   
31        M6H      West Toronto   
32        M6J      West Toronto   
33        M6K      West Toronto   
34        M6P      West Toronto   
35        M6R      West Toronto   
36        M6S      West Toronto   
37        M7Y      East Toronto   

                                        Neighborhoods 1st Most Common Venue  \
0                                         The Beaches          Neighborhood   
1                        The Danforth West, Riverdale      Greek Restaurant   
2                      The Beaches West, India Bazaar                  Park   
3                                     Studio District                  Café   
4                                       Lawrence Park                  Park   
5                                    Davisville North           Pizza Place   
6                                  North Toronto West        Clothing Store   
7                                          Davisville        Sandwich Place   
8                         Moore Park, Summerhill East            Playground   
9   Deer Park, Forest Hill SE, Rathnelly, South Hi...           Coffee Shop   
10                                           Rosedale                  Park   
11                        Cabbagetown, St. James Town           Coffee Shop   
12                               Church and Wellesley           Coffee Shop   
13                          Harbourfront, Regent Park           Coffee Shop   
14                           Ryerson, Garden District           Coffee Shop   
15                                     St. James Town           Coffee Shop   
16                                        Berczy Park           Coffee Shop   
17                                 Central Bay Street           Coffee Shop   
18                           Adelaide, King, Richmond           Coffee Shop   
19  Harbourfront East, Toronto Islands, Union Station           Coffee Shop   
20           Design Exchange, Toronto Dominion Centre           Coffee Shop   
21                     Commerce Court, Victoria Hotel           Coffee Shop   
22                                           Roselawn                Garden   
23                Forest Hill North, Forest Hill West                  Park   
24                The Annex, North Midtown, Yorkville                  Café   
25                     Harbord, University of Toronto                  Café   
26          Chinatown, Grange Park, Kensington Market                  Café   
27  CN Tower, Bathurst Quay, Island airport, Harbo...      Airport Terminal   
28                    Stn A PO Boxes 25 The Esplanade           Coffee Shop   
29             First Canadian Place, Underground city           Coffee Shop   
30                                           Christie         Grocery Store   
31                       Dovercourt Village, Dufferin              Pharmacy   
32                           L

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 0, 2, 2, 2, 4, 2])

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = toronto_new.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(38, 16)


PostalCode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               2          Neighborhood     Health Food Store   
1 -79.352188               2      Greek Restaurant           Coffee Shop   
2 -79.315572               2                  Park        Sandwich Place   
3 -79.340923               2                  Café           Coffee Shop   
4 -79.388790               0                  Park                Lawyer   

  3rd Most Common Venue 4th Most Common Venue   5th Most Common Venue  \
0  Other Great Outdoors                 Trail                     Pub   
1        Ice Cream Shop    Italian Restaurant  Furniture / Home Store   
2          Intersection        Ice Cream Shop                 Brewery   
3    Italian Restaurant                Bakery     American Restaurant   
4              Bus Line           Swim School             Yoga Studio   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0   Dumpling Restaurant        Discount Store               Dog Run   
1             Bookstore               Brewery       Bubble Tea Shop   
2         Burrito Place          Liquor Store  Fast Food Restaurant   
3    Chinese Restaurant           Fish Market             Bookstore   
4               Dog Run  Fast Food Restaurant        Farmers Market   

  9th Most Common Venue 10th Most Common Venue  
0      Doner Restaurant             Donut Shop  
1                  Café             Restaurant  
2     Fish & Chips Shop      Food & Drink Shop  
3               Brewery     Seafood Restaurant  
4    Falafel Restaurant            Event Space

In [33]:

# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(38, 16)


PostalCode           Borough  \
4         M4N   Central Toronto   
23        M5P   Central Toronto   
10        M4W  Downtown Toronto   
0         M4E      East Toronto   
21        M5L  Downtown Toronto   
24        M5R   Central Toronto   
25        M5S  Downtown Toronto   
26        M5T  Downtown Toronto   
27        M5V  Downtown Toronto   
28        M5W  Downtown Toronto   
29        M5X  Downtown Toronto   
30        M6G  Downtown Toronto   
31        M6H      West Toronto   
32        M6J      West Toronto   
33        M6K      West Toronto   
34        M6P      West Toronto   
35        M6R      West Toronto   
20        M5K  Downtown Toronto   
19        M5J  Downtown Toronto   
18        M5H  Downtown Toronto   
17        M5G  Downtown Toronto   
1         M4K      East Toronto   
2         M4L      East Toronto   
3         M4M      East Toronto   
5         M4P   Central Toronto   
6         M4R   Central Toronto   
7         M4S   Central Toronto   
36        M6S      West Toronto   
9         M4V   Central Toronto   
12        M4Y  Downtown Toronto   
13        M5A  Downtown Toronto   
14        M5B  Downtown Toronto   
15        M5C  Downtown Toronto   
16        M5E  Downtown Toronto   
11        M4X  Downtown Toronto   
37        M7Y      East Toronto   
22        M5N   Central Toronto   
8         M4T   Central Toronto   

                                         Neighborhood   Latitude  Longitude  \
4                                       Lawrence Park  43.728020 -79.388790   
23                Forest Hill North, Forest Hill West  43.696948 -79.411307   
10                                           Rosedale  43.679563 -79.377529   
0                                         The Beaches  43.676357 -79.293031   
21                     Commerce Court, Victoria Hotel  43.648198 -79.379817   
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678   
25                     Harbord, University of Toronto  43.662696 -79.400049   
26          Chinatown, Grange Park, Kensington Market  43.653206 -79.400049   
27  CN Tower, Bathurst Quay, Island airport, Harbo...  43.628947 -79.394420   
28                    Stn A PO Boxes 25 The Esplanade  43.646435 -79.374846   
29             First Canadian Place, Underground city  43.648429 -79.382280   
30                                           Christie  43.669542 -79.422564   
31                       Dovercourt Village, Dufferin  43.669005 -79.442259   
32                           Little Portugal, Trinity  43.647927 -79.419750   
33       Brockton, Exhibition Place, Parkdale Village  43.636847 -79.428191   
34                      High Park, The Junction South  43.661608 -79.464763   
35                             Parkdale, Roncesvalles  43.648960 -79.456325   
20           Design Exchange, Toronto Dominion Centre  43.647177 -79.381576   
19  Harbourfront East, Toronto Islands, Union Station  43.640816 -79.381752   
18                           Adelaide, King, Richmond  43.650571 -79.384568   
17                                 Central Bay Street  43.657952 -79.387383   
1                        The Danforth West, Riverdale  43.679557 -79.352188   
2                      The Beaches West, India Bazaar  43.668999 -79.315572   
3                                     Studio District  43.659526 -79.340923   
5                                    Davisville North  43.712751 -79.390197   
6                                  North Toronto West  43.715383 -79.405678   
7                                          Davisville  43.704324 -79.388790   
36                                 Runnymede, Swansea  43.651571 -79.484450   
9   Deer Park, Forest Hill SE, Rathnelly, South Hi...  43.686412 -79.400049   
12                               Church and Wellesley  43.665860 -79.383160   
13                          Harbourfront, Regent Park  43.654260 -79.360636   
14                           Ryerson, Garden District  43.657162 -79.378937   
15                            

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
4  Central Toronto               0                  Park   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
4                Lawyer              Bus Line           Swim School   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
4           Yoga Studio               Dog Run  Fast Food Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
4        Farmers Market    Falafel Restaurant            Event Space

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
23   Central Toronto               1                  Park   
10  Downtown Toronto               1                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23                 Trail         Jewelry Store      Sushi Restaurant   
10              Building            Playground                 Trail   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23           Yoga Studio        Discount Store        Farmers Market   
10           Yoga Studio               Dog Run      Doner Restaurant   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
23    Falafel Restaurant           Event Space         Ethiopian Restaurant  
10            Donut Shop   Dumpling Restaurant  Eastern European Restaurant

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0       East Toronto               2          Neighborhood   
21  Downtown Toronto               2           Coffee Shop   
24   Central Toronto               2                  Café   
25  Downtown Toronto               2                  Café   
26  Downtown Toronto               2                  Café   
27  Downtown Toronto               2      Airport Terminal   
28  Downtown Toronto               2           Coffee Shop   
29  Downtown Toronto               2           Coffee Shop   
30  Downtown Toronto               2         Grocery Store   
31      West Toronto               2              Pharmacy   
32      West Toronto               2                   Bar   
33      West Toronto               2           Coffee Shop   
34      West Toronto               2                  Park   
35      West Toronto               2        Breakfast Spot   
20  Downtown Toronto               2           Coffee Shop   
19  Downtown Toronto               2           Coffee Shop   
18  Downtown Toronto               2           Coffee Shop   
17  Downtown Toronto               2           Coffee Shop   
1       East Toronto               2      Greek Restaurant   
2       East Toronto               2                  Park   
3       East Toronto               2                  Café   
5    Central Toronto               2           Pizza Place   
6    Central Toronto               2        Clothing Store   
7    Central Toronto               2        Sandwich Place   
36      West Toronto               2                  Café   
9    Central Toronto               2           Coffee Shop   
12  Downtown Toronto               2           Coffee Shop   
13  Downtown Toronto               2           Coffee Shop   
14  Downtown Toronto               2           Coffee Shop   
15  Downtown Toronto               2           Coffee Shop   
16  Downtown Toronto               2           Coffee Shop   
11  Downtown Toronto               2           Coffee Shop   
37      East Toronto               2           Yoga Studio   

            2nd Most Common Venue 3rd Most Common Venue  \
0               Health Food Store  Other Great Outdoors   
21                          Hotel                  Café   
24                 Sandwich Place           Coffee Shop   
25                     Restaurant                   Bar   
26  Vegetarian / Vegan Restaurant                   Bar   
27                Airport Service              Boutique   
28                           Café            Restaurant   
29                           Café                 Hotel   
30                           Café                  Park   
31                         Bakery           Supermarket   
32                    Coffee Shop           Men's Store   
33                 Breakfast Spot                  Café   
34                           Café    Mexican Restaurant   
35                      Gift Shop          Dessert Shop   
20                           Café                 Hotel   
19                       Aquarium                 Hotel   
18                           Café                   Bar   
17             Italian Restaurant                  Café   
1                     Coffee Shop        Ice Cream Shop   
2                  Sandwich Place          Intersection   
3                     Coffee Shop    Italian Restaurant   
5                             Gym        Breakfast Spot   
6             Sporting Goods Shop           Coffee Shop   
7                    Dessert Shop           Pizza Place   
36                    Pizza Place    Italian Restaurant   
9                             Pub   American Restaurant   
12               Sushi Restaurant   Japanese Restaurant   
13                           Café                   Pub   
14                 Clothing Store        Cosmetics Shop   
15                           Café    Italian Restaurant   
16                   Cocktail Bar    Seafood Restaurant   
11              

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
22  Central Toronto               3                Garden   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
22           Yoga Studio        Discount Store  Fast Food Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
22        Farmers Market    Falafel Restaurant           Event Space   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
22  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
8  Central Toronto               4            Playground   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
8                   Gym           Summer Camp           Yoga Studio   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
8                 Diner        Farmers Market    Falafel Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
8           Event Space  Ethiopian Restaurant      Electronics Store


Observations:

Most of the neighborhoods fall into Cluster 1 which are mostly business areas with cafe, restaurants, supermarkets etc. Cluster 2 is just a garden, Cluster 3 are playground and park, Cluster 4 park and swim school, and lastly Cluster 5 park and trail.